In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
#Progress bar fix: use callbacks=[Logger.JupyterProgbarLogger()] in model.fit
#verbose=0 is also required
import JupyterProgbarLogger as Logger
from tqdm import tqdm_notebook as tqdm
from kerastuner.tuners import RandomSearch

import sklearn.metrics as metrics

import numpy as np
import random
import math
import matplotlib.pyplot as plt

import h5py

###FIX NUMPY LOAD FOR DICTIONARIES\
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [2]:
BATCH_SIZE=32
DATA_AMOUNT=300000

In [3]:
def sort_data(data,labels,train_len,val_len,test_len,blocksize=1):
    curBlock = labels[0]
    train_data = np.zeros((train_len,80,80,1))
    val_data = np.zeros((val_len,80,80,1))
    test_data = np.zeros((test_len,80,80,1))
    train_labels = np.full(train_len,-1)
    val_labels = np.full(val_len,-1)
    test_labels = np.full(test_len,-1)
    train_ind = 0
    val_ind = 0
    test_ind = 0
    choices = np.arange(3)
    i=0
    while i < len(data):
        start = i
        iter_amt = len(data) if blocksize == 1 else blocksize
        while labels[i]==curBlock and i-start < iter_amt:
            i+=1
            if i == len(data):
                break
        end = i
        if not i == len(data):
            curBlock = labels[i]
        if not blocksize == 1 and end-start < blocksize:
            continue
        cur_frames = data[start:end]
        cur_labels = labels[start:end]
        choice=-1
        if choices.size>0:
            choice = np.random.choice(choices)
        else:
            break
        if choice == 0:
            if train_ind + len(cur_frames) >= train_len:
                cur_frames = cur_frames[:train_len-train_ind]
                cur_labels = cur_labels[:train_len-train_ind]
                choices=np.delete(choices,np.argwhere(choices==0))
            train_data[train_ind:train_ind+len(cur_frames)]=cur_frames
            train_labels[train_ind:train_ind+len(cur_frames)]=cur_labels
            train_ind += len(cur_frames)
        elif choice == 1:
            if val_ind + len(cur_frames) >= val_len:
                cur_frames = cur_frames[:val_len-val_ind]
                cur_labels = cur_labels[:val_len-val_ind]
                choices=np.delete(choices,np.argwhere(choices==1))
            val_data[val_ind:val_ind+len(cur_frames)]=cur_frames
            val_labels[val_ind:val_ind+len(cur_frames)]=cur_labels
            val_ind += len(cur_frames)
        elif choice == 2:
            if test_ind + len(cur_frames) >= test_len:
                cur_frames = cur_frames[:test_len-test_ind]
                cur_labels = cur_labels[:test_len-test_ind]
                choices=np.delete(choices,np.argwhere(choices==2))
            test_data[test_ind:test_ind+len(cur_frames)]=cur_frames
            test_labels[test_ind:test_ind+len(cur_frames)]=cur_labels
            test_ind += len(cur_frames)
    return train_data[:train_ind-(train_ind%blocksize)],train_labels[:train_ind-(train_ind%blocksize)],val_data[:val_ind-(val_ind%blocksize)],val_labels[:val_ind-(val_ind%blocksize)],test_data[:test_ind-(test_ind%blocksize)],test_labels[:test_ind-(test_ind%blocksize)]

In [4]:
file_path = "images_raw_doric_round1.h5"
with h5py.File(file_path,'r') as f:
    labels = f['/labels'][:DATA_AMOUNT]
    data = f['/frames/raw'][:DATA_AMOUNT]
data = data[(labels>=0)]
labels = labels[(labels>=0)]
DATA_AMOUNT=len(data)
if len(data.shape) < 4:
    data=data[...,None]
tl = math.floor(DATA_AMOUNT/2)
vl=tsl=math.floor(DATA_AMOUNT/4)
depth=5
train_data,train_labels,val_data,val_labels,test_data,test_labels = sort_data(data,labels,tl,vl,tsl,blocksize=depth)

In [9]:
def build_model(
                input_shape=(1,80, 80, 1),
                stride_length=(1, 1),
                kernel=(4, 4),
                kernel_initializer='glorot_uniform',
                activation=layers.Activation('relu'),
                dense_activation=layers.Activation('relu'),
                output_activation=layers.Activation('softmax'),
                batch_momentum=.999,
                combine=True,
                padding='valid',
                batch_norm=False
            ):
    name = "3D CNN"
    nfilters=[64,128,256]
    inputs = layers.Input(shape=input_shape)
    x = inputs


    conv_parameters = {
        'padding': padding,
        'strides': stride_length,
        'kernel_initializer': kernel_initializer
    }


    # encode net
    for filters in nfilters:
        x = layers.Conv3D(filters, kernel, **conv_parameters)(x)
        x = layers.BatchNormalization(momentum=batch_momentum)(x)
        x = activation(x)
        x = layers.MaxPooling3D((2, 2, 2), padding=padding)(x)
    #x = layers.Conv2D(1, (1, 1))(x)
    #x = activation(x)
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(46)(x)
    output = output_activation(x)


    model = keras.models.Model(inputs, output)
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=1e-4,
            momentum=0.9,
            nesterov=True,
            decay=1e-6
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model,name

In [11]:
if not depth == 1:
    model,name = build_model(input_shape=(depth,80,80,1),stride_length=(1,1,1),kernel=(4,4,4))
    train_data=np.reshape(train_data,(len(train_data)/depth,depth,80,80,1))
    val_data=np.reshape(val_data,(len(val_data)/depth,depth,80,80,1))
    test_data=np.reshape(test_data,(len(test_data)/depth,depth,80,80,1))
    train_labels=train_labels[::depth]
    val_labels=val_labels[::depth]
    test_labels=test_labels[::depth]
else:
    model,name = build_model()

ValueError: Negative dimension size caused by subtracting 4 from 1 for 'conv3d_6/Conv3D' (op: 'Conv3D') with input shapes: [?,1,38,38,64], [4,4,4,64,128].

In [ ]:
history = model.fit(train_data,
                    train_labels,
                    epochs=10,
                    verbose=0,
                    batch_size=BATCH_SIZE,
                    validation_data=(val_data,val_labels),
                    callbacks=[Logger.JupyterProgbarLogger()]
                   )

In [ ]:
def accuracy(predictions, answers):
    accs = np.zeros(len(answers))
    for i in range(0,len(answers)):
        if(predictions[i]==answers[i]):
            accs[i]=1
    return np.mean(accs)

In [ ]:
predicts = np.argmax(model.predict(test_data),1)

In [ ]:
#plt.hist((test_labels,predicts), density=True, histtype='bar', stacked=True, label = ("Data","Predictions"))
#plt.legend()
plt.plot(test_labels)
plt.plot(predicts)
plt.xlim(0,100)
plt.show()

In [ ]:
print(accuracy(predicts,test_labels))
print(metrics.f1_score(test_labels,predicts,average=None))

In [ ]:
model_benchmarks = np.load("benchmarks.npy")
model_benchmarks.item()[name]=metrics.f1_score(test_labels,predicts,average=None)
np.save("benchmarks.npy",model_benchmarks)
print(model_benchmarks)

In [ ]:
print(train_data.shape)

In [ ]:
print(test_labels[0:300])

In [ ]:
plt.plot(labels)
ind = 500
plt.xlim(ind,ind+10)
plt.show()